In [1]:
import torch,torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from model import Generator, iterate_minibatches, compute_loss, train
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
import time
import torch.utils.data
from tqdm import tqdm
from generate import write_file, generate
import pickle

/opt/miniconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from train_models import LstmDiscriminator, FeaturedLstmDiscriminator, LstmBaseline
from scores import EB, UPC, QN

In [3]:
def g_loss(p_fake,sound,in_probs,p_baseline, eps = 1e-8):
        
    probs = sound[:,:,:,:2]*in_probs[:,:,:,:2]\
            +(1-sound[:,:,:,:2])*(1-in_probs[:,:,:,:2])
        
    return -((probs+eps).log().sum(dim =-1).sum(dim =-1).sum(dim =-1)*(p_fake-p_baseline)).mean(), probs

def g_loss_monte_carlo(p_fakes,sounds,in_probs,p_baseline, eps = 1e-8):
    
    loss = 0
    for sound, p_fake in zip(sounds, p_fakes):
        probs = sound[:,:,:,:2]*in_probs[:,:,:,:2]\
                +(1-sound[:,:,:,:2])*(1-in_probs[:,:,:,:2])
        loss += -((probs+eps).log().sum(dim =-1).sum(dim =-1).sum(dim =-1)*(p_fake-p_baseline)).mean()
        
    return loss/5, probs

def d_loss(p_fake, p_true,eps = 1e-8):
    return -(1-p_fake+eps).log().mean()-(p_true+eps).log().mean()
    
def bl_loss(bl_pred,real_reward):
    return (bl_pred-real_reward).pow(2).mean()


In [4]:
import torch.utils.data

def sample_sound(data_gen):
    size = data_gen.size()
    rand = torch.rand(*size).cuda()
    sample = (rand<data_gen).type(torch.FloatTensor).cuda()
    sample[:,:,:,2] = sample[:,:,:,0]
    return sample


In [5]:
OCTAVE_NUM = 4
NOTE_NUM = 12
TIME_SCALE = 128

from dataset import load_all
from constants import *

In [6]:
styles= [['data/Bach']]
X_tr, y_tr = load_all(styles, BATCH_SIZE, SEQ_LEN)
print(X_tr.shape, y_tr.shape)

X_loader = torch.utils.data.DataLoader(\
            torch.utils.data.TensorDataset(\
            *(torch.FloatTensor(X_tr),
            torch.FloatTensor(y_tr))),\
            batch_size=8,shuffle=True)

(5727, 128, 48, 3) (5727, 128, 48, 3)


In [7]:
def train_discriminator(num_epochs = 3,d_lr = 0.001, verbose = True):
    
    X_loader = torch.utils.data.DataLoader(\
            torch.utils.data.TensorDataset(\
            *(torch.FloatTensor(X_tr),
            torch.FloatTensor(y_tr))),\
            batch_size=8,shuffle=True)

    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=d_lr, betas=(0.5, 0.999)) #, weight_decay = 1, betas=(0.5, 0.999))  
    
    history = {'d_losses':[], 'prop_machine':[],
               'prob_human':[], 'prob_noise':[], 'color':[]}

    generator.eval()
    generator.note_ax.to_train=True
    generator.note_ax.apply_T=False
        
    for epoch in range(num_epochs):        
        start_time = time.time()
        for i, [x_batch,ch_batch] in tqdm(enumerate(X_loader)):
            try:
                x_batch = x_batch.cuda()
                ch_batch = ch_batch.cuda()
                
                #x_batch[:,:,:,2] = x_batch[:,:,:,0] #1 #x_batch[:,:,:,0]
                ch_batch[:,:,:,2] = ch_batch[:,:,:,0] # 1 #ch_batch[:,:,:,0]

                # Optimize D  
                # rand_like ones_like            
                noise_example = sample_sound(torch.rand_like(x_batch).cuda()), x_batch
                true_example = ch_batch , x_batch
                p_true_example = discriminator(*true_example)
                p_noise_example = discriminator(*noise_example)
                loss = d_loss(p_noise_example, p_true_example)
                d_optimizer.zero_grad()
                loss.backward()
                d_optimizer.step()            

                # machine generation
                data_gen, sound = generator(x_batch,ch_batch)
#                 sound = sample_sound(data_gen).data          
                false_example = sound, x_batch
                true_example = ch_batch, x_batch
                p_true_example = discriminator(*true_example)
                p_false_example = discriminator(*false_example)
                loss = d_loss(p_false_example, p_true_example)
                
                d_optimizer.zero_grad()
                loss.backward()
                d_optimizer.step()
                
                # history
                history['d_losses'].append(loss.data.cpu().numpy())
                history['prop_machine'].append(p_false_example.mean().data.cpu().numpy())
                history['prob_human'].append(p_true_example.mean().data.cpu().numpy())
                history['prob_noise'].append(p_noise_example.mean())
                
            except KeyboardInterrupt:
                return history, True

    #       # Visualize
        if verbose:
            display.clear_output(wait=True)
            plt.figure(figsize=(16, 6))

            # Then we print the results for this epoch:
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time)) 
            print('mean probability of human generation: {}'.format(history['prob_human'][-1]))
#             print('mean probability of noise generation: {}'.format(prob_noise[-1]))
            print('mean probability of machine generation: {}'.format(history['prop_machine'][-1]))

            plt.title("losses")
            plt.subplot(121)
            plt.plot(history['prob_human'],label = "probability of human generation")
            plt.plot(history['prop_machine'],label = "probability of machine generation")
            plt.legend()
            plt.subplot(122)
            plt.plot(history['d_losses'],label = "Discriminator loss")
            plt.legend()
            plt.show()
                
            
    return history, False

In [8]:
# h= train_discriminator( num_epochs = 1, d_lr=1e-3)

In [9]:
def train_generator(num_epochs = 3,g_lr = 0.001, verbose=True, monte_carlo=True, 
                    entropy_loss = True, teacher_forcing = 2):
    
    g_optimizer = torch.optim.Adam(generator.parameters(),  lr=g_lr, betas=(0.5, 0.999))#, betas=(0.5, 0.999))
    
    history = {'g_losses':[],'prop_machine':[],
               'prob_human':[],'prob_confidence':[],'prob_noise':[],
                  'EB_true':[], 'UPC_true':[], 'QN_true':[], 'EB_false':[], 'UPC_false':[], 'QN_false':[]}
    
    generator.train()
    generator.note_ax.to_train=True
    generator.note_ax.apply_T=False
    
    X_loader = torch.utils.data.DataLoader(\
            torch.utils.data.TensorDataset(\
            *(torch.FloatTensor(X_tr),
            torch.FloatTensor(y_tr))),\
            batch_size=3,shuffle=True)

    for epoch in range(num_epochs):
        start_time = time.time()
        for i, [x_batch,ch_batch] in tqdm(enumerate(X_loader)):
            
            try:
                
                x_batch = x_batch.cuda()
                ch_batch = ch_batch.cuda()
                
                #x_batch[:,:,:,2] = x_batch[:,:,:,0] #1 #x_batch[:,:,:,0]
                ch_batch[:,:,:,2] = ch_batch[:,:,:,0] #1 #ch_batch[:,:,:,0]

                # baseline
                p_baseline = baseline(x_batch,ch_batch)

                #generator
                # monte carlo false example sampling
                data_gen, sound = generator(x_batch,ch_batch)
                if monte_carlo:
                    p_false_examples = []
                    sounds = []

                    for _ in range(5):                  
    #                     sound = sample_sound(data_gen).data                 
                        false_example = sound.detach(), x_batch.detach()
                        p_false_example = discriminator(*false_example)
                        p_false_examples.append(p_false_example)
                        sounds.append(sound)
                else:
    #                 sound = sample_sound(data_gen).data                 
                    false_example = sound.detach(), x_batch.detach()
                    p_false_example = discriminator(*false_example)


                true_example = ch_batch.detach(), x_batch.detach()
                p_true_example = discriminator(*true_example)

                # Optimize G  
                handle = np.random.randint(0,teacher_forcing-1)
#                 handle = 0
                if handle != 0:
                    if monte_carlo:
                        loss, confidence = g_loss_monte_carlo(p_false_examples,
                                  sounds,data_gen,p_baseline)

                    else:
                        loss, confidence = g_loss(p_false_example,
                                  sound,data_gen,p_baseline)

                else:
                    # teaher forcing
                    loss, confidence = g_loss(1,
                                  ch_batch,data_gen,p_baseline)

                if entropy_loss:
                    loss+=compute_loss(data_gen, ch_batch)   
                    
                g_optimizer.zero_grad()
                loss.backward()
                g_optimizer.step()
                
            except KeyboardInterrupt:
                return history, True
        
            # history
            history['g_losses'].append(loss.data.cpu().numpy())
            history['prob_human'].append(p_true_example.mean().data.cpu().numpy())
            history['prop_machine'].append(p_false_example.mean().data.cpu().numpy())
            history['prob_confidence'].append(confidence.mean().data.cpu().numpy())
            history['EB_true'].append(EB(ch_batch).mean().data.cpu().numpy())
            history['UPC_true'].append(UPC(ch_batch).mean().data.cpu().numpy())
            history['QN_true'].append(QN(ch_batch).mean().data.cpu().numpy())
            history['EB_false'].append(EB(sound).mean().data.cpu().numpy())
            history['UPC_false'].append(UPC(sound).mean().data.cpu().numpy())
            history['QN_false'].append(QN(sound).mean().data.cpu().numpy())

        # Visualize
        if verbose:
            display.clear_output(wait=True)
            plt.figure(figsize=(16, 6))

            # Then we print the results for this epoch:
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time)) 
            print('mean probability of human generation: {}'.format(history['prob_human'][-1]))
            print('mean probability of machine generation: {}'.format(history['prop_machine'][-1]))
    #         print('mean probability of noise generation: {}'.format(prob_noise[-1]))
            print('mean probability of machine generation confidence: {}'.format(history['prob_confidence'][-1]))


            plt.title("losses")
            plt.subplot(231)
            plt.plot(history['prob_human'], label = "Human probability")
            plt.plot(history['prop_machine'], label = "Machine probability")
            plt.legend()
            plt.subplot(232)
            plt.plot(history['g_losses'], label = "Generator loss")
            plt.legend()
            plt.subplot(233)
            plt.plot(history['prob_confidence'],label = "Machine confidence")
            plt.legend()
            plt.subplot(234)
            plt.plot(history['EB_true'],label = "EB_true")
            plt.plot(history['EB_false'],label = "EB_false")
            plt.legend()
            plt.subplot(235)
            plt.plot(history['UPC_true'],label = "UPC_true")
            plt.plot(history['UPC_false'],label = "UPC_false")
            plt.legend()
            plt.subplot(236)
            plt.plot(history['QN_true'],label = "QN_true")
            plt.plot(history['QN_false'],label = "QN_false")
            plt.legend()
            plt.show()
  
    return history, False

In [10]:
# train_generator(num_epochs = 100, g_lr=1e-3, verbose = True)

In [34]:
def train_baseline(num_epochs = 3, b_lr = 0.001, verbose=True):
    
    bl_optimizer = torch.optim.Adam(baseline.parameters(),  lr=b_lr)
    
    bl_losses = []
    prob_baseline = []     
    generator.eval()
    
    for epoch in range(num_epochs):
        start_time = time.time()
        for i, [x_batch,ch_batch] in tqdm(enumerate(X_loader)):
            
            x_batch = x_batch.cuda()
            ch_batch = ch_batch.cuda()
            
            ch_batch[:,:,:,2] = ch_batch[:,:,:,0]
            
            # Optimize BL
            data_gen, sound = generator(x_batch,ch_batch)
#             sound = sample_sound(data_gen).data
            false_example = sound.detach(), x_batch.detach()
            p_baseline = baseline(x_batch,ch_batch)
            p_false_example = discriminator(*false_example)
            loss = bl_loss(p_baseline, p_false_example)
            
            bl_optimizer.zero_grad()
            loss.backward()
            bl_optimizer.step()
            
            # history
            bl_losses.append(loss.data.cpu().numpy())
            prob_baseline.append(p_baseline.mean().data.cpu().numpy())

            
    return np.array(prob_baseline),np.array(bl_losses)

In [35]:
# discriminator = FeaturedLstmDiscriminator(hidden_size=1000,last_dim=78*2).cuda()
# generator = Generator().cuda()
# generator.load_state_dict(torch.load(os.path.join(OUT_DIR, 'model_canonical_attention')))

In [36]:
# prob_human, prop_machine, d_losses = train_discriminator( num_epochs = 40, d_lr=1e-3)

In [37]:
# torch.save(discriminator.state_dict(), os.path.join(OUT_DIR, 'discriminator'))

In [38]:
# generator.load_state_dict(torch.load(os.path.join(OUT_DIR, 'model_canonical')))
# baseline = LSTM_baseline(hidden_size=10).cuda()
# generator,prob_human, prop_machine, prob_confidence, g_losses= train_generator(generator, discriminator, baseline, X_loader, num_epochs = 30, g_lr=1e-3)

In [39]:
PATH = '/data/i.anokhin/'

In [63]:
def plot_history(history):
    
    plt.figure(figsize=(16, 6))
    plt.title("losses")
    plt.subplot(331)
    plt.plot(history['g_losses'],label = "Generator loss")
    plt.legend()
    plt.subplot(332)
    plt.plot(history['d_losses'],label = "Discriminator loss")
    plt.legend(loc=2)
    plt.subplot(333)
    plt.plot(history['bl_losses'],label = "Baseline loss")
    plt.legend()
    plt.subplot(334)
    g_machine = np.asarray(history['prop_machine'].copy())
    d_machine = np.asarray(history['prop_machine'].copy())
    g_machine[np.asarray(history['color']) == 1] = np.nan
    d_machine[np.asarray(history['color']) == 0] = np.nan
    plt.plot(g_machine, 'o', label = "Machine P - generator", color = 'blue' )
    plt.plot(d_machine, 'o', label = "Machine P - discriminator", color = 'orange' )
    plt.plot(history['prob_human'], 'o', label = "Human P", color = 'green')
    plt.legend(loc=2)
    plt.subplot(335)
    plt.plot(history['prob_confidence'],label = "Machine confidence")
    plt.legend()
    plt.subplot(336)
    plt.plot(history['prop_baseline'],label = "Baseline probabilyty")
    plt.legend()
    plt.subplot(337)
    plt.plot(history['EB_true'],label = "EB_true")
    plt.plot(history['EB_false'],label = "EB_false")
    plt.legend()
    plt.subplot(338)
    plt.plot(history['UPC_true'],label = "UPC_true")
    plt.plot(history['UPC_false'],label = "UPC_false")
    plt.legend()
    plt.subplot(339)
    plt.plot(history['QN_true'],label = "QN_true")
    plt.plot(history['QN_false'],label = "QN_false")
    plt.legend()
    plt.show()
    
    
def train_GAN(g_lr = 0.001, d_lr = 0.001, bl_lr = 0.001, history = None, criterion=True, 
              verbose=True, dump_history=True):
    
    if not history:     
        history = {'d_losses':[],'g_losses':[],'bl_losses':[],'prop_machine':[],
               'prob_human':[],'prob_confidence':[1,],'prop_baseline':[], 'prob_noise':[], 'color':[],
                  'EB_true':[], 'UPC_true':[], 'QN_true':[], 'EB_false':[], 'UPC_false':[], 'QN_false':[]}
    
    epoch = 0
    n_epochs = 5000
    it_generator = 0
    it_discriminator = 0

    while epoch<n_epochs:
        start_time = time.time()
        epoch+=1
        
        try:  
            
            #if ((epoch % 1) ==0):
            if dump_history:
                    pickle.dump(history, open(PATH+'rl_history.p', "wb" ))

              
            if criterion:
                print('discriminator trainnig')
                if it_generator>0:
                    write_file('output/rl_test_'+str(epoch), generate(generator, 32, to_train=True))
                    torch.save(generator.state_dict(), PATH+'generator_'+str(epoch))
                    
                it_generator = 0
                it_discriminator+=1
                
                generator.eval()
                h, flag = train_discriminator(num_epochs = 1, d_lr=d_lr, verbose = False)
                
                for k in h.keys():
                    history[k].extend(h[k])
                history['color'].extend(np.ones(len(h['prob_human'])))
                
            else: 
                print('generator trainnig')
                if it_discriminator>0:
                    torch.save(discriminator.state_dict(), PATH+'discriminator_'+str(epoch))
                    
                it_discriminator = 0
                
                p_baseline, bl_l = train_baseline(num_epochs = 1,b_lr = bl_lr, verbose=False)
                history['prop_baseline'].extend(p_baseline)
                history['bl_losses'].extend(bl_l) 

                it_generator+=1
                generator.train()
                h, flag = train_generator(num_epochs = 3, g_lr=g_lr, verbose = False, 
                                    monte_carlo = True, teacher_forcing=2, entropy_loss = True)
                
                for k in h.keys():
                    history[k].extend(h[k])
                history['color'].extend(np.zeros(len(h['prob_human'])))
                
            p_human = np.array(history['prob_human'][-100:])
            p_machine =  np.array(history['prop_machine'][-100:])
            diff = np.maximum(p_human-p_machine,0).mean()
            criterion = (diff<0.1) or (p_human.mean()<0.8 and diff<0.3)
#             and it_generator>5
#             or (p_human.mean()<0.9 and diff<0.3)
            if flag:
                return epoch, history
                
        except KeyboardInterrupt:
            return epoch, history  
                    
        
        if verbose:
            # Visualize
            display.clear_output(wait=True)
            # Then we print the results for this epoch:
            if history['color'][-1]==0:
                tr = 'generator trainig'
            else: tr = 'last trainig - discriminator '
            print("Epoch {} of {} took {:.3f}s, format {}".format(
                epoch, n_epochs, time.time() - start_time, tr))
            print('mean probability of machine generation: {}'.format(history['prop_machine'][-1]))
            print('mean probability of human generation: {}'.format(history['prob_human'][-1]))
        #         print('mean probability of noise generation: {}'.format(prob_noise[-1]))
            print('mean probability of machine generation confidence: {}'.format(history['prob_confidence'][-1]))

            plot_history(history)
            
    return epoch, history

In [64]:
# epoch = 5
# write_file('output/rl_test_'+str(epoch), generate(generator, 32, to_train=True))

In [65]:
# discriminator = FeaturedLstmDiscriminator(hidden_size=1000,last_dim=78*2).cuda()
# generator = Generator().cuda()
# baseline = LstmBaseline(hidden_size=40).cuda()

# generator.load_state_dict(torch.load(os.path.join(OUT_DIR, 'model_test2'))) #model_test2 enerator_rl
# discriminator.load_state_dict(torch.load(os.path.join(OUT_DIR, 'discriminator')))

In [66]:
# discriminator.load_state_dict(torch.load(os.path.join(OUT_DIR, 'discriminator')))
# generator.note_ax.to_train=True
# generator.note_ax.apply_T=False
# generator.train()

In [67]:
# h= train_discriminator( num_epochs = 20, d_lr=1e-3)

In [68]:
# p_baseline, bl_l = train_baseline(num_epochs = 1,b_lr = 0.001, verbose=False)

In [69]:
# generator = Generator().cuda()
# generator.load_state_dict(torch.load(os.path.join(OUT_DIR, 'model_canonical_attention')))
# generator.train()
# h  = train_generator(num_epochs = 1, g_lr=1e-3, verbose = True, monte_carlo = False, 
#                      teacher_forcing=3, entropy_loss = True)

In [70]:
# discriminator = FeaturedLstmDiscriminator(hidden_size=1000,last_dim=78*2).cuda()
# generator = Generator().cuda()
# baseline = LstmBaseline(hidden_size=40).cuda()

In [71]:
# torch.save(generator.state_dict(), '/data/i.anokhin/generator')
# torch.save(discriminator.state_dict(), '/data/i.anokhin/discriminator')
# torch.save(baseline.state_dict(), '/data/i.anokhin/baseline')
# generator.load_state_dict(torch.load('/data/i.anokhin/generator'))
# discriminator.load_state_dict(torch.load('/data/i.anokhin/discriminator'))
# baseline.load_state_dict(torch.load('/data/i.anokhin/baseline'))

In [72]:
# generator.load_state_dict(torch.load(os.path.join(OUT_DIR, 'model_test2')))
history = None

In [ ]:
generator.train()
generator.note_ax.to_train=True
generator.note_ax.apply_T=False
epoch, history = train_GAN(g_lr = 1e-3,d_lr=1e-3, bl_lr = 1e-3, history = history, criterion=True, verbose=False)

discriminator trainnig


114it [00:51,  2.21it/s]

In [32]:
# plot_history(history)

In [94]:
# generator.time_ax.use_beat
generator.note_ax.to_train=True
generator.note_ax.apply_T=False
# import gc
# torch.cuda.empty_cache()

generator.eval()
# with torch.cuda.device(GPU):
write_file('output/rl_test', generate(generator, 32, to_train=True))

Generating with no styles:


NameError: name 'time_ax' is not defined